In [2]:
from torchvision.models import resnet50
import torch
import matplotlib.pyplot as plt
import numpy as np

In [3]:
saved_name = 'dzmodel.pt'
device_2 = torch.device('cpu')
model_2 = resnet50()
model_2.fc = torch.nn.Sequential(
    torch.nn.Linear(2048, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 74)
)
model_2.load_state_dict(torch.load(saved_name, map_location=device_2))


/tmp/ipykernel_2181/2457755700.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_2.load_state_dict(torch.load(saved_name, map_location=device_2))


<All keys matched successfully>

In [4]:
model_2.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
example_input = torch.rand(1,3,224,224)
traced_model = torch.jit.trace(model_2, example_input)
out = traced_model(example_input)

In [6]:
import coremltools as ct
print(ct.__version__)

scale = 1/(0.226*255.0)
bias = [- 0.485/(0.229) , - 0.456/(0.224), - 0.406/(0.225)]

# mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225]

# print(bias)

Torch version 2.4.1 has not been tested with coremltools. You may run into unexpected errors. Torch 2.2.0 is the most recent version that has been tested.
Failed to load _MLModelProxy: No module named 'coremltools.libcoremlpython'


7.2


In [7]:

classifier_config = ct.ClassifierConfig("labels.txt")
image_input = ct.ImageType(name="input_1", shape=example_input.shape, scale=scale, bias=bias)
core_ml_model = ct.convert(
    traced_model,
    convert_to="mlprogram",
    inputs=[image_input],
    classifier_config=classifier_config
)

Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 253.39 passes/s]


In [8]:
# Save the converted model.
core_ml_model.save("DetroitZoo1.mlpackage")